In [12]:
import random
import itertools
import numpy as np
from collections import namedtuple

import gym
import tensorflow

from MCTS import MCTS, PongEnv, Node
from utils import partition_points

%load_ext autoreload
%reload_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


__Selection Phase__

Starting from root node, run *Tree Policy*

*Tree Policy*

Cases:
1. Not terminal and no children --> run __Expansion__; proceed to case 2
2. Not terminal and unexplored children --> Select from unexplored children, proceed to __Simulation__
3. Not terminal and all children explored --> Apply bandit algo to select action / child.  Step forward env then select on child.
4. Terminal --> Break

__Expansion__
Initialize child nodes, one for each action

__Simulation__
Run trajectory according to *rollout policy* starting from node state until terminal state is reached, then __backprop__

Pong-specific: end simulation when a point is awarded not according to Pong environment terminal signal, which returns true only at end of game (when one player has reached 21 points).  

__Backprop__

Starting from terminal node, update value and visits recursively, applying discounts as needed, until root.  

In [2]:
#Create env
pong = PongEnv()

#Initial State
init_state = pong.state
root = Node(state=init_state)

#Run MCTS
mcts = MCTS()

[2016-09-14 03:23:29,928] Making new env: Pong-v0


In [3]:
mcts.select(root, pong)

expanding

exploring child 1
simulating
Num steps:  1647
Final Score: 3 to 21
Num Up moves: 815
Num Down moves: 832

backprop'ing



In [13]:
penv = PongEnv()

[2016-09-14 03:30:55,879] Making new env: Pong-v0


In [14]:
results = penv.simulate()

In [15]:
s, a, r = results

In [17]:
seqs = partition_points(s, a, r)

In [20]:
s0 = seqs[0]
s1 = seqs[1]

In [22]:
init_state0 = s0[0][0]

In [24]:
init_state0.shape

(210, 160, 3)

In [25]:
init_state_1 = s1[0][0]

In [26]:
init_state_1.shape

(210, 160, 3)

In [27]:
np.allclose(init_state0, init_state_1)

False